## Strategy pattern:

Strategy lets you define a family of algorithms, put each of them into a separate class, and make their objects interchangeable at runtime. 

In [1]:
# A car object has drive method. However, it may have multiple driving strategies during a journey

class Car(object):
    
    def __init__(self, driver=None):
        self._strategy = driver
        
    def set_driver(self, driver):
        self._strategy = driver
        
    def drive(self):
        if self._strategy is None:
            raise AttributeError('No driver!')
        self._strategy.drive()
        
        
class Alan(): 
    """Strategy A: Driver Alan"""
    def drive(self):
        print('Alan is driving Smoothly!')

        
class Bob():
    """Strategy B: Driver Bob"""
    def drive(self):
        print('Bob is driving Crazy!')


# The stategies can be instantiated using factory
class Driver(object):
    
    @staticmethod
    def factory(name):
        if name.lower() == "alan":
            return Alan()
        elif name.lower() == "bob":
            return Bob()
        else:
            print(f"{name} is not available!")

            
if __name__ == "__main__":
    car = Car()
    #name = input("Select a driver: ")
    name = 'Alan'
    driver = Driver.factory(name)
    car.set_driver(driver)
    car.drive()

Alan is driving Smoothly!


#### Applicability:
  * Use the Strategy pattern when you want to use different variants of an algorithm within an object and be able to switch from one algorithm to another during runtime.
  * Use the pattern to isolate the business logic of a class from the implementation details of algorithms that may not be as important in the context of that logic.

## State pattern
State is a behavioral design pattern that lets an object alter its behavior when its internal state changes. It appears as if the object changed its class.

The main idea is that, at any given moment, there’s a finite number of states which a program can be in. Within any unique state, the program behaves differently, and the program can be switched from one state to another instantaneously.

In [2]:
# Example: When you drive a autonomic car, the next state of the car depends on its current state.

class Car(object):
    
    def __init__(self, state):
        self._state = state
    
    @property
    def speed(self):
        return f'Moving Speed {self._state._speed}'

    def speedup(self):
        print("Speeding up")
        self._state = self._state.speedup_state()
    
    def slowdown(self):
        print("Slowing down")
        self._state = self._state.slowdown_state()

class State():
    
    def speedup_state(self):
        raise NotImplemented
        
    def slowdown_state(self):
        raise NotImplemented
        
class Stop(State):
    
    _speed = '0 km/h'
    
    def speedup_state(self):
        return DriveSlow()
    
    def slowdown_state(self):
        return Stop()

class DriveSlow(State):
    
    _speed = '40 km/h'
    
    def speedup_state(self):
        return DriveNormal()
    
    def slowdown_state(self):
        return Stop()
    
class DriveNormal(State):
    
    _speed = '60 km/h'
    
    def speedup_state(self):
        return DriveFast()
    
    def slowdown_state(self):
        return DriveSlow()
    
class DriveFast(State):
    
    _speed = '100 km/h'
    
    def speedup_state(self):
        return DriveFast()
    
    def slowdown_state(self):
        return DriveNormal()

    
car = Car(Stop())

if __name__ == '__main__':
    print("=== Speed up the car ===")
    for i in range(4):
       car.speedup()
       print(car.speed)

    print('\n')
    
    print("=== Slow down the car ===")
    for i in range(4):
       car.slowdown()
       print(car.speed)

=== Speed up the car ===
Speeding up
Moving Speed 40 km/h
Speeding up
Moving Speed 60 km/h
Speeding up
Moving Speed 100 km/h
Speeding up
Moving Speed 100 km/h


=== Slow down the car ===
Slowing down
Moving Speed 60 km/h
Slowing down
Moving Speed 40 km/h
Slowing down
Moving Speed 0 km/h
Slowing down
Moving Speed 0 km/h


## Command
Command is a behavioral design pattern that turns a request into a stand-alone object that contains all information about the request, i.e. the receiver and parameter. The invoker only need to execute the commands rather than to handle the complex interfaces of receivers.

### How to implement
Four terms always associated with the command pattern are command, receiver, invoker and client.
  * A command object knows about receiver and invokes a method of the receiver. Values for parameters of the receiver method are stored in the command. The receiver object to execute these methods is also stored in the command object by aggregation.
  * The receiver then does the work when the execute() method in command is called.
  * An invoker object knows how to execute a command, and optionally does bookkeeping about the command execution. The invoker does not know anything about a concrete command, it knows only about the command interface.
  * Invoker objects, command objects and receiver objects are held by a client object, the client decides which receiver objects it assigns to the command objects, and which commands it assigns to the invoker. The client decides which commands to execute at which points. To execute a command, it passes the command object to the invoker object.

In [3]:
from abc import ABC, abstractmethod


class Command(ABC):
    """
    The Command interface declares a method for executing a command.
    """
    @abstractmethod
    def execute(self):
        pass


class ConcreteCommandA(Command):
    """
    Concrete command
    """

    def __init__(self, receiver):
        """
        commands can accept one or several receiver objects along with
        any parameters for receivers via the constructor.
        """

        self._receiver = receiver
        self._param = 'Parament A'

    def execute(self) -> None:
        """
        Commands can delegate to any methods of a receiver.
        """
        self._receiver.operation_a(self._param)
        
        
class ConcreteCommandB(Command):
    """
    Concrete command
    """

    def __init__(self, receiver):
        """
        commands can accept one or several receiver objects along with
        any parameters for receivers via the constructor.
        """

        self._receiver = receiver
        self._param = 'param B'

    def execute(self) -> None:
        """
        Commands can delegate to any methods of a receiver.
        """
        self._receiver.operation_b(self._param)


class Receiver:
    """
    The Receiver classes contain some important business logic. They know how to
    perform all kinds of operations, associated with carrying out a request. In
    fact, any class may serve as a Receiver.
    """

    def operation_a(self, param):
        print("Operation A with {}".format(param))

    def operation_b(self, param):
        print("Operation B with {}".format(param))


class Invoker:
    """
    The Invoker is associated with one or several commands. It sends a request
    to the command.
    """

    def start(self):
        print("Doing some at the start")

    def finish(self):
        print("Doing some at the end")

    def do_your_work(self, command):
        """
        The Invoker does not depend on concrete command or receiver classes. The
        Invoker passes a request to a receiver indirectly, by executing a
        command.
        """
        self.start()
        command.execute()
        self.finish()


if __name__ == "__main__":
    """
    The client code can parameterize an invoker with any commands.
    """
    receiver = Receiver()
    command = ConcreteCommandA(receiver)
    invoker = Invoker()
    invoker.do_your_work(command)

    receiver = Receiver()
    invoker = Invoker()
    command = ConcreteCommandB(receiver)
    invoker.do_your_work(command)

Doing some at the start
Operation A with Parament A
Doing some at the end
Doing some at the start
Operation B with param B
Doing some at the end


In [4]:
# from collections import deque


class Switch(object):
    """The INVOKER class"""
    def __init__(self):
        self._history = deque()

    @property
    def history(self):
        return self._history

    def execute(self, command):
        self._history.appendleft(command)
        command.execute()

        
class Command(object):
    """The COMMAND interface"""
    def __init__(self, obj):
        self._obj = obj

    def execute(self):
        raise NotImplementedError

        
class TurnOnCommand(Command):
    """The COMMAND for turning on the light"""
    def execute(self):
        self._obj.turn_on()

        
class TurnOffCommand(Command):
    """The COMMAND for turning off the light"""
    def execute(self):
        self._obj.turn_off()

        
class Light(object):
    """The RECEIVER class"""
    def turn_on(self):
        print("The light is on")

    def turn_off(self):
        print("The light is off")

        
class LightSwitchClient(object):
    """The CLIENT class"""
    def __init__(self):
        self._lamp = Light()
        self._switch = Switch()

    @property
    def switch(self):
        return self._switch

    def press(self, cmd):
        cmd = cmd.strip().upper()
        if cmd == "ON":
            self._switch.execute(TurnOnCommand(self._lamp))
        elif cmd == "OFF":
            self._switch.execute(TurnOffCommand(self._lamp))
        else:
            print("Argument 'ON' or 'OFF' is required.")


# Execute if this file is run as a script and not imported as a module
if __name__ == "__main__":
    light_switch = LightSwitchClient()
    
    light_switch.press("ON")

    light_switch.press("OFF")
    light_switch.press("****")

    print(light_switch.switch.history)

NameError: name 'deque' is not defined

### Note:
Command and Strategy may look similar because you can use both to parameterize an object with some action. However, they have very different intents:
  * The Strategy Pattern usually describes different ways of doing the same thing. The strategy object decide how to implement the work.
  * The Command Pattern converts any operation into an object. The command stores the receiver and parameters for receiver. The receiver decide how to implement the work. The invoker object will only need to work on individual command objects but not to handle the complexity of receivers.

## Visitor

Visitor is a behavioral design pattern that lets you separate algorithms from the objects on which they operate.

### How to implement:
1. Declare the visitor interface with a set of “visiting” methods, one per each concrete element class that exists in the program. each visiting method accepts an element as the argument.

2. Add accept interface to elements. This method accepts a visitor object as an argument.

4. Now, any concrete element object with accept interface is able to accept any concrete visitor method and direct its operation to visitors.

In [ ]:
class Element():
    """
    The Component interface declares an `accept` method that should take the
    base visitor interface as an argument.
    """
    def accept(self, visitor):
        pass


class ConcreteElementA(Element):
    def accept(self, visitor):
        """
        Note that we're calling `visitConcreteComponentA`, which matches the
        current class name. This way we let the visitor know the class of the
        component it works with.
        """
        visitor.visit_concrete_element_a(self)

    def operation_a(self):
        """
        Concrete Components may have special methods that don't exist in their
        base class or interface. The Visitor is still able to use these methods
        since it's aware of the component's concrete class.
        """
        return "Operation A"

class ConcreteElementB(Element):
    def accept(self, visitor):
        visitor.visit_concrete_element_b(self)

    def operation_b(self):
        return "Operation B"
    

class Visitor():
    """
    The Visitor Interface declares a set of visiting methods that correspond to
    component classes. The signature of a visiting method allows the visitor to
    identify the exact class of the component that it's dealing with.
    """
    def visit_concrete_element_a(self, element: ConcreteElementA):
        pass
    
    def visit_concrete_element_b(self, element: ConcreteElementB):
        pass


class ConcreteVisitor1(Visitor):
    def visit_concrete_element_a(self, element):
        print(f"{element.operation_a()} from ConcreteVisitor1")
        
    def visit_concrete_element_b(self, element):
        print(f"{element.operation_b()} from ConcreteVisitor1")


class ConcreteVisitor2(Visitor):
    def visit_concrete_element_a(self, element):
        print(f"{element.operation_a()} from ConcreteVisitor2")
        
    
    def visit_concrete_element_b(self, element):
        print(f"{element.operation_b()} from ConcreteVisitor2")


if __name__ == "__main__":
    element = ConcreteElementA()
    visitor = ConcreteVisitor1()
    element.accept(visitor)

    element = ConcreteElementA()
    visitor = ConcreteVisitor2()
    element.accept(visitor)
    
    element = ConcreteElementB()
    visitor = ConcreteVisitor1()
    element.accept(visitor)

    element = ConcreteElementB()
    visitor = ConcreteVisitor2()
    element.accept(visitor)

In [ ]:
"""
Visitor pattern example. Credit: https://en.wikipedia.org/wiki/Visitor_pattern
"""

from abc import ABCMeta, abstractmethod

NOT_IMPLEMENTED = "You should implement this."

class CarElement:
    __metaclass__ = ABCMeta
    @abstractmethod
    def accept(self, visitor):
        raise NotImplementedError(NOT_IMPLEMENTED)

class Body(CarElement):
    def accept(self, visitor):
        visitor.visitBody(self)


class Engine(CarElement):
    def accept(self, visitor):
        visitor.visitEngine(self)


class Wheel(CarElement):
    def __init__(self, name):
        self.name = name
    def accept(self, visitor):
        visitor.visitWheel(self)

class Car(CarElement):
    def __init__(self):
        self.elements = [
            Wheel("front left"), Wheel("front right"),
            Wheel("back left"), Wheel("back right"),
            Body(), Engine()
        ]

    def accept(self, visitor):
        for element in self.elements:
            element.accept(visitor)
        visitor.visitCar(self)

class CarElementVisitor:
    __metaclass__ = ABCMeta
    @abstractmethod
    def visitBody(self, element):
        raise NotImplementedError(NOT_IMPLEMENTED)
    @abstractmethod
    def visitEngine(self, element):
        raise NotImplementedError(NOT_IMPLEMENTED)
    @abstractmethod
    def visitWheel(self, element):
        raise NotImplementedError(NOT_IMPLEMENTED)
    @abstractmethod
    def visitCar(self, element):
        raise NotImplementedError(NOT_IMPLEMENTED)

class CarElementDoVisitor(CarElementVisitor):
    def visitBody(self, body):
        print("Moving my body.")
    def visitCar(self, car):
        print("Starting my car.")
    def visitWheel(self, wheel):
        print("Kicking my {} wheel.".format(wheel.name))
    def visitEngine(self, engine):
        print("Starting my engine.")


class CarElementPrintVisitor(CarElementVisitor):
    def visitBody(self, body):
        print("Visiting body.")
    def visitCar(self, car):
        print("Visiting car.")
    def visitWheel(self, wheel):
        print("Visiting {} wheel.".format(wheel.name))
    def visitEngine(self, engine):
        print("Visiting engine.")

car = Car()
car.accept(CarElementPrintVisitor())
car.accept(CarElementDoVisitor())

### Chain of responsibility
Chain of Responsibility is a behavioral design pattern that lets you pass requests along a chain of handlers. Upon receiving a request, each handler decides either to process the request or to pass it to the next handler in the chain.

In [ ]:
class Handler(object):
    """
    The Handler interface declares a method for building the chain of handlers.
    It also declares a method for executing a request.
    """

    _next_handler = None

    def set_next(self, handler):
        print(handler)
        self._next_handler = handler
        return handler

    def handle(self, request):
        if self._next_handler:
            return self._next_handler.handle(request)


class MonkeyHandler(Handler):
    def handle(self, request):
        if request == "Banana":
            return f"Monkey: I'll eat the {request}"
        else:
            return super().handle(request)


class SquirrelHandler(Handler):
    def handle(self, request):
        if request == "Nut":
            return f"Squirrel: I'll eat the {request}"
        else:
            return super().handle(request)


class DogHandler(Handler):
    def handle(self, request):
        if request == "MeatBall":
            return f"Dog: I'll eat the {request}"
        else:
            return super().handle(request)


def client_code(handler: Handler):
    """
    The client code is usually suited to work with a single handler. In most
    cases, it is not even aware that the handler is part of a chain.
    """

    for food in ["Nut", "Banana", "Cup of coffee"]:
        print(f"\nClient: Who wants a {food}?")
        result = handler.handle(food)
        if result:
            print(f"  {result}", end="")
        else:
            print(f"  {food} was left untouched.", end="")


if __name__ == "__main__":
    monkey = MonkeyHandler()
    squirrel = SquirrelHandler()
    dog = DogHandler()

    monkey.set_next(squirrel).set_next(dog)

    # The client should be able to send a request to any handler, not just the
    # first one in the chain.
    print("Chain: Monkey > Squirrel > Dog")
    client_code(monkey)
    print("\n")

    print("Subchain: Squirrel > Dog")
    client_code(squirrel)

### Observer pattern:

Observer is a behavioral design pattern that lets you define a subscription mechanism to notify multiple objects about any events that happen to the object they’re observing.

The object that has some interesting state is often called subject, The subject is also called it publisher since it’s going to notify other objects about the changes to its state. All other objects that want to track changes to the publisher’s state are called subscribers or observers.

Implementation:
  1. Add a subscription mechanism to the publisher:
    * An list field for storing a list of subscribers
    * Several public methods which allow adding and removing subscribers from that list.
    * A notify method for calling subscribers' update method to send notification messages
  2. Add update method to observers. The update method defines the reaction to the notification.

The publisher class doesn’t have to maintain the subscription list by itself. It can delegate this job to the special helper object.

In [ ]:
from abc import ABC, abstractmethod
import time


class Subject(ABC):


    @abstractmethod
    def add_observer(self, observer):
        pass

    @abstractmethod
    def remove_observer(self, observer):
        pass

    @abstractmethod
    def notify(self):
        """
        Notify all observers about an event.
        """
        pass


class ConcreteSubject(Subject):

    _state: str = ""

    # Listst of subscribers. In real life, the list of subscribers can be stored more comprehensively (categorized by event type, etc.).
    _observers = []

    def add_observer(self, observer):
        self._observers.append(observer)

    def remove_observer(self, observer):
        self._observers.remove(observer)

    def notify(self):
        """
        Trigger an update in each subscriber.
        """
        for observer in self._observers:
            observer.update(self._state)

    def get_current_time(self) -> None:
        self._state = time.asctime()
        self.notify()


class Observer(ABC):
    """
    The Observer interface declares the update method, used by subjects.
    """

    @abstractmethod
    def update(self, time):
        """
        Receive update from subject.
        """
        pass


class ConcreteObserverA(Observer):
    
    def update(self, time):
        print('Observer A is calibrated to time: {}'.format(time))


class ConcreteObserverB(Observer):
    def update(self, time):
        print('Observer B is calibrated to time: {}'.format(time))

        
if __name__ == "__main__":
    # The client code.

    subject = ConcreteSubject()

    observer_a = ConcreteObserverA()
    subject.add_observer(observer_a)

    print('Notifying Oberver A ...')
    subject.get_current_time()
    
    observer_b = ConcreteObserverB()
    subject.add_observer(observer_b)

    print('Notifying Oberver A and B...')
    subject.get_current_time()

## Mediator

Mediator is a behavioral design pattern that lets you reduce chaotic dependencies between objects. The pattern restricts direct communications between the objects and forces them to collaborate only via a mediator object.

Implementation:
  * Create a mediator:
   1. Declare the mediator interface and describe the desired communication protocol between mediators and various components.
   2. The mediator should store the references to components.
  * For components:
   3. Components should store a reference to the mediator object.
   4. A component will call mediator's notification method instead of methods on other components for communication.

In [ ]:
from abc import ABC


class Mediator(ABC):
    """
    The Mediator interface declares a method used by components to notify the
    mediator about various events. The Mediator may react to these events and
    pass the execution to other components.
    """

    def notify(self, sender, event):
        pass


class ConcreteMediator(Mediator):
    def __init__(self, component1, component2):
        self._component1 = component1
        self._component1.mediator = self
        self._component2 = component2
        self._component2.mediator = self

    def notify(self, sender, event):
            if sender == 'component 1':
                if event == 'event A':
                    self._component2.react_a()
                if event == "event B":
                    self._component2.react_b()

class BaseComponent:
    """
    The Base Component provides the basic functionality of storing a mediator's
    instance inside component objects.
    """

    def __init__(self, mediator=None):
        self._mediator = mediator

    @property
    def mediator(self):
        return self._mediator

    @mediator.setter
    def mediator(self, mediator):
        self._mediator = mediator


class Component1(BaseComponent):
    
    def do_a(self):
        print("Component 1 does A.")
        self.mediator.notify("component 1", 'event A')

    def do_b(self):
        print("Component 1 does B.")
        self.mediator.notify("component 1", 'event B')


class Component2(BaseComponent):
    def react_a(self):
        print("Component 2 reacts A.")

    def react_b(self):
        print("Component 2 reacts B.")


if __name__ == "__main__":
    # The client code.
    c1 = Component1()
    c2 = Component2()
    mediator = ConcreteMediator(c1, c2)

    print("Client triggers operation A.")
    c1.do_a()

    print("\n", end="")

    print("Client triggers operation B.")
    c1.do_b()

### Thank You